In [1]:
from os import listdir
from os.path import isfile, join
from astropy.io import fits
import numpy as np

#making bias master
onlyfiles = [join('bias_dark', f) for f in listdir('bias_dark') if isfile(join('bias_dark', f))]
frames=np.array([fits.open(f) for f in onlyfiles])
zero_imgs=[f[0].data for f in frames if f[0].header['IMAGETYP'] == 'BIAS']
zero_cube=np.dstack(zero_imgs)
zero_master=np.median(zero_cube,axis=2)

In [2]:
#makeing dark master
from collections import Counter
dark_exp=Counter([f[0].header['EXPTIME'] for f in frames if f[0].header['IMAGETYP'] == 'DARK'])
all_exp=[f[0].header['EXPTIME'] for f in frames]# if f[0].header['IMAGETYP'] == 'DARK']

dark_index=[np.where(np.array(all_exp) == k) for k in dark_exp]

master_dk={}
for dktime in dark_exp.keys():
    dk_imgs=np.dstack([f[0].data for f in frames if (f[0].header['IMAGETYP'] == 'DARK') & (f[0].header['EXPTIME'] == dktime)])
    dk_med=np.median(dk_imgs,axis=2)
    if dktime not in master_dk:
        master_dk[dktime]=dk_med


In [3]:
# making flat master
flatfiles = [join('AutoFlat', f) for f in listdir('AutoFlat') if isfile(join('AutoFlat', f))]
flatframes=np.array([fits.open(f) for f in flatfiles])
flat_exp=Counter([f[0].header['FILTER'] for f in flatframes if f[0].header['IMAGETYP'] == 'FLAT'])
allflat_exp=[f[0].header['FILTER'] for f in flatframes]# if f[0].header['IMAGETYP'] == 'DARK']
flat_index=[np.where(np.array(allflat_exp) == k) for k in flat_exp]


master_flat={}
for fname in flat_exp.keys():
    flat_imgs=np.dstack([(f[0].data-zero_master)/np.mean(f[0].data-zero_master) for f in flatframes if (f[0].header['IMAGETYP'] == 'FLAT') & (f[0].header['FILTER'] == fname)])
    flat_med=np.median(flat_imgs,axis=2)
    if fname not in master_flat:
        master_flat[fname]=flat_med

In [4]:
objfiles = [join('pschiang', f) for f in listdir('pschiang') if isfile(join('pschiang', f))]
objframes=np.array([fits.open(f) for f in objfiles])

obj_exp=[f[0].header['EXPOSURE'] for f in objframes]
objheaders = [f[0].header for f in objframes]

In [5]:
for f in objframes:
    pheader=f[0].header
    reduced_img=(f[0].data - master_dk[f[0].header['EXPTIME']])/master_flat[f[0].header['FILTER']]
    pheader.set('BZERO',0)
    pheader['comment'] = 'reduced by Poshih python pipeline'
    outfits = fits.PrimaryHDU(data=reduced_img,header=pheader)
    outfits.writeto('r_'+f[0].fileinfo()['file'].name.split('/')[1])

In [ ]:
objframes[0][0].fileinfo()['file'].name.split('/')[1]